In [3]:
import pandas as pd
df = pd.read_pickle('kbl_preprocessed.pkl')

In [5]:
from bkcharts import Histogram
from bkcharts import defaults, show, output_file
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [9]:
defaults.width = 600
defaults.height = 350

# input options
hist = Histogram(pd.DataFrame(df[df['sen_length']<3000]['sen_length']), title="df['sen_length']", bins=100)
show(hist)

In [10]:
df=df[df['sen_length']>25]
df=df.reset_index(drop=True)

In [11]:
df.head(5)

,level_0,contents,datetime,title,url,label_num,sentences,sen_length,con_length
0,HH,[KBO리그] 전체적 전력 미흡으로 가을야구 실패했지만...[오마이뉴스 김승훈 기자...,2015-10-03 20:43:03,아쉬운 마리한화... 그러나 박수가 아깝지 않다,http://sports.news.naver.com/kbaseball/news/re...,1,전체적 전력 미흡으로 가을야구 실패했지만. 한화 이글스가 2009년부터 2014...,3393,3562
1,HH,(서울=연합뉴스) 천병혁 기자 = 신생팀 케이티 위즈가 통렬한 홈런포를 쏘아 올리며...,2015-10-03 16:47:21,"케이티, 신생팀 최다승 타이 한화 PS 좌절",http://sports.news.naver.com/kbaseball/news/re...,1,신생팀 케이티 위즈가 통렬한 홈런포를 쏘아 올리며 한화 이글스의 실낱같은 희망마저...,838,987
2,HH,한화 이글스 선수단. 2015.9.30 뉴스1 News1 신성룡 기자(서울=뉴스...,2015-10-03 16:50:42,"[프로야구]화제의 중심 한화, 결국 포스트시즌 탈락",http://sports.news.naver.com/kbaseball/news/re...,1,한화 이글스 선수단. 2015.9. 숱한 화제를 뿌렸던 한화 이글스의 2015시즌이...,2273,2386
3,HH,한화 이글스 선수단. (연합뉴스 자료사진) 2008년 이후 가장 높은 승률 흥행 성...,2015-10-03 16:57:10,성공과 실패의 경계선 위에 선 2015년 한화,http://sports.news.naver.com/kbaseball/news/re...,1,한화 이글스 선수단. 2015 KBO리그 중심에 김성근 감독의 한화 이글스가 있었다...,1423,1671
4,HH,[엑스포츠뉴스=이지은 기자] 후회는 이미 늦다. 하지만 분명 아쉬운 순간들은 남았다...,2015-10-03 17:00:04,[PS 진출실패] 한화에게 뼈아팠던 아쉬운 세 경기,http://sports.news.naver.com/kbaseball/news/re...,1,후회는 이미 늦다. 하지만 분명 아쉬운 순간들은 남았다. 한화 이글스는 3일 수...,2099,2165


In [12]:
X = df.sentences
y = df.label_num
print(X.shape)
print(y.shape)

(196993,)
(196993,)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(147744,)
(49249,)
(147744,)
(49249,)


In [15]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 94.8 ms


In [16]:
stopwords_kbo=['조','동안','때','구단','만','월','일','메이저','타율','실점','등판','명','치','원','피','프로','이날','상대','타선','연승','연패','상황','연속','이후','후','루','뒤','등','년','팀','승','것','월','위','패','전','번','군','개','점','차','중','기사','제공','팬','루수','회','말','초','야구','리그']

### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [17]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_kbo, ngram_range=(1,3), min_df=2, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 9min 21s, sys: 5.7 s, total: 9min 26s
Wall time: 9min 27s


In [18]:
X_test_dtm = vect.transform(X_test)

In [19]:
X_train_dtm.shape

(147744, 3095372)

In [20]:
X_test_dtm.shape

(49249, 3095372)

## save dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python

In [21]:
import pickle

with open("kbo_tfidf.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [22]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)

In [23]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 2.52 s, sys: 256 ms, total: 2.78 s
Wall time: 2.78 s


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [24]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [25]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.94284147901480231

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['두산', '한화', 'LG', '넥센', '기아', '삼성', 'NC', '롯데', 'SK', 'KT']))

             precision    recall  f1-score   support

         두산       0.94      0.95      0.95      7479
         한화       0.95      0.96      0.96      5976
         LG       0.94      0.94      0.94      5125
         넥센       0.94      0.94      0.94      5192
         기아       0.93      0.94      0.94      4857
         삼성       0.92      0.94      0.93      4540
         NC       0.95      0.93      0.94      4405
         롯데       0.95      0.95      0.95      4415
         SK       0.95      0.92      0.94      4345
         KT       0.94      0.93      0.94      2915

avg / total       0.94      0.94      0.94     49249



In [17]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=0) & (y_test==0)])
print(np.argmax(counts))

5


In [24]:
X_test[(y_pred_class==5) & (y_test==0)]

114235    이재용 삼성 부회장, 홍라희 삼성미술관장, 이서현삼성물산 사장 등 삼성 일가가 29...
117936      삼성은 강했고 두산은 버티지 못하며 끝내 역전패 당했다.삼성 라이온즈는 18일 ...
120172      삼성이 전 두산 내야수 최영진(28)을 영입했다.속초상고와 한일장신대를 거쳐 2...
120443      정든 팀을 떠나야 하는 FA 보상선수. 그러나 꼭 서운한 일만은 아니다. 새로운...
114718    프로야구 두산의 한국시리즈 우승을 이끈 김현수 등 역대 최대인 22명의  자유계약선...
120492    삼성 라이온즈가 자유계약선수(FA) 시장에서 내야수 이원석(30)을 영입했다. 삼성...
116484     19일 오후 서울 송파구 잠실야구장에서 열린 2015 프로야구 두산 베어스와 삼성...
120624    이원석의 합류로 포지션 경쟁이 치열해질 삼성 내야.   연합뉴스내야수 이원석이 두산...
120652    삼성 김한수 감독은 이원석의 영입을 적극적으로 원했다. 내야 무한 경쟁의 신호탄이 ...
134512    1일 오후 대구 삼성라이온즈파크에서 열리는  2017 프로야구  삼성 라이온즈와 두...
112455     차우찬.  삼성은 4차전 선발로 알프레도 피가로를 내세웠다. 두산의 4차전 선발은...
116314    김태형 두산 감독과 양의지. 스포츠조선 DB.두산 안방마님 양의지(28)는 2007...
123455    2016 프로야구 한국시리즈 1차전 NC와 두산의 경기가 29일 서울 잠실야구장에서...
113033    25일 대구경북디자인센터에서 2015 타이어뱅크 KBO리그 한국시리즈 미디어데이가 ...
120495    내야수 이원석이 삼성에 새로운 둥지를 튼다.    내야수 이원석(30)이 삼성 라이...
110269    26일 오후 대구시민야구장에서 열린 2015 KBO 한국시리즈 1차전 삼성 라이온즈...
128178    두산 베어스 장원준은 지난해까지 통산 97승을 올려 올

In [25]:
X_test[114235]

'이재용 삼성 부회장, 홍라희 삼성미술관장, 이서현삼성물산 사장 등 삼성 일가가 29일 서울 잠실야구장에서 열린 한국시리즈 3차전 두산과 삼성 경기를 관람하고 있다. 2015.10.29 연합뉴스 삼성 라이온즈가 2경기 연속 1득점에 그친 타선의 침묵과 결정적 수비 실책 탓에 두산 베어스에 무릎을 꿇었다. 7전4승제의 한국시리즈에서 1승 2패로 열세에 몰린 삼성은 30일 열리는 4차전을 이겨야 하는 부담을 안게 됐다. 통산 8번째 한국시리즈 우승을 노리는 삼성이 먼저 2패를 당하고도 챔피언 반지를 차지한 것은 두산과 맞붙었던 2013년이 유일하다. 삼성은 29일 서울 잠실야구장에서 열린 두산 베어스와의 한국시리즈 3차전에서 8안타를 쳤으나 응집력 부족으로 1대5로 패했다. 1회 내야안타를 치고 나간 구자욱이 나바로의 적시타로 홈을 밟은 게 유일한 득점이었다. 삼성은 앞서 27일 대구시민야구장에서 열린 2차전에서도 1대6으로 졌다.가을비에 젖은 삼성 타자들의 방망이는 두산 선발투수 장원준의 변화구 앞에 물 먹은 솜방망이처럼 무거웠다. 3 5회에는 9타자 연속 범타에 그쳤고, 6회 2사 2 3루, 8회 무사 1루, 9회 2사 만루에서는 적시타가 터지지 않았다. 멀티 히트를 때려낸 것은 한국시리즈 첫 선발 출장의 기회를 잡은 구자욱과 이지영뿐이었다. 삼성 선발투수 타일러 클로이드는 5이닝 5피안타 6사사구로 3실점, 자신의 첫 한국시리즈 등판에서 패전의 멍에를 썼다. 매 이닝 주자를 내보내고도 3회까지는 무실점으로 버텼으나 4회 볼넷 2개로 자초한 1사 2 3루에서 박건우에게 2타점 적시타를 맞았다. 또 5회에는 2루타와 몸에 맞는 공, 희생번트에 이은 양의지의 희생플라이로 추가점을 내줬다. 삼성은 6회 1사 만루의 위기에서 구원투수 심창민이 허경민을 2루수 앞 땅볼로 유도했으나 2루수 나바로가 1루에 악송구를 하는 바람에 한꺼번에 2점을 헌납, 분위기를 완전히 넘겨주고 말았다.'

In [26]:
df[df.index==114235]

,level_0,contents,datetime,title,url,label_num,sentences,sen_length,con_length
114235,OB,"이재용 삼성 부회장, 홍라희 삼성미술관장, 이서현삼성물산 사장 등 삼성 일가가 29...",2015-10-29 23:50:04,"타선 침묵 수비 실책 잇따라 삼성, KS 3차전 두산에 1대5로 져",http://sports.news.naver.com/kbaseball/news/re...,0,"이재용 삼성 부회장, 홍라희 삼성미술관장, 이서현삼성물산 사장 등 삼성 일가가 29...",949,1010


In [27]:
df[df.index==123417]

,level_0,contents,datetime,title,url,label_num,sentences,sen_length,con_length
123417,OB,"[일간스포츠 유병민] 2016시즌 프로야구 최고의 순간 주인공으로 이승엽(삼성),...",2016-12-12 20:50:59,카스모멘트 정재훈 고통보다 아웃 잡겠다는 생각 뿐이었다,http://sports.news.naver.com/kbaseball/news/re...,0,"2016시즌 프로야구 최고의 순간 주인공으로 이승엽(삼성), 정재훈(두산), ...",1072,1149


In [28]:
X_test[123417]

'   2016시즌 프로야구 최고의 순간 주인공으로 이승엽(삼성), 정재훈(두산), 황덕균(넥센)이 꼽혔다.  카스포인트 어워즈가 12일 서울 삼성동 코엑스 오디토리움에서 열렸다. 카스포인트 어워즈는 누적 점수에 따라 순위를 결정하는 카스포인트 부문과 팀 승리에 결정적 영향을 미친 역동적인 장면과 감동적인 모습을 선정하는 카스모멘트 부문으로 구분해서 시상한다. 영예의 대상은 투 타 카스포인트 상위 4명 가운데 카스포인트(40 )와 선정위원회 점수(10 ), 네티즌 현장 투표(50 )를 합산해 선정한다. 카스모멘트는 10개 부문 중 선정위원 점수(20 )와 네티즌 현장 투표(80 )를 합산해서 베스트 3를 선정했다.  카스모멘트 베스트 3 는 한 일 통산 600홈런의 주인공 이승엽과 팔꿈치 부상을 당하면서도 송구를 시도한 정재훈, 프로 데뷔 15년 만에 첫 승을 따낸 황덕균이 선정됐다. 정재훈은 지난 8월4일 잠실 LG전에서 4 5로 뒤진 8회 2사 1 2루 위기에서 박용택의 타구에 팔꿈치를 강타 당했다. 부상 속에서도 1루 송구를 위해 왼손으로 공을 잡았지만, 끝내 던지지 못하고 그라운드에 주저 앉았다. 팔꿈치 골절 부상을 당한 정재훈은 팀이 2연패에 성공한 한국시리즈에 나서지 못해 아쉬움을 남겼다. 시즌을 마친 뒤 어깨 수술을 해 내년 시즌 초반 등판이 불투명하다. 정재훈은  지금까지 부상과 수술없이 야구를 했는데, 최근 어깨 수술까지 두 차례 수술을 했다 며  재활까지 시간이 걸릴 것 같다. 재활을 잘 마무리해서 좋은 공을 던지겠다 고 소감을 밝혔다.  팔꿈치 부상 당시 상황을 회상한 정재훈은  몸에 가해진 고통보다 어떻게든 아웃카운트를 잡아야 한다는 생각 뿐이었다. 왼손으로 잘 던져서 던지려고 했는데, 타이밍이 맞지 않았다. 고통은 그 다음에 오더라. 어느 투수라도 나와 같은 행동을 했을 것 이라며 웃었다. 이어  내년 초반 재활로 시작할 것 같다. 내년에는 3연패를 하는데 있어서 마지막에 동료와 같이 있을 수 있도록, 마운드에서 공 던질 수 있도록 준비

In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [28]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [29]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 46.2 s, sys: 100 ms, total: 46.3 s
Wall time: 46.3 s


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [30]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [31]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.96706532112327159

In [32]:
print(classification_report(y_test, y_pred_class_svm, target_names=['두산', '한화', 'LG', '넥센', '기아', '삼성', 'NC', '롯데', 'SK', 'KT']))

             precision    recall  f1-score   support

         두산       0.97      0.97      0.97      7479
         한화       0.97      0.98      0.98      5976
         LG       0.96      0.96      0.96      5125
         넥센       0.96      0.97      0.97      5192
         기아       0.96      0.96      0.96      4857
         삼성       0.96      0.96      0.96      4540
         NC       0.97      0.97      0.97      4405
         롯데       0.97      0.97      0.97      4415
         SK       0.97      0.97      0.97      4345
         KT       0.96      0.96      0.96      2915

avg / total       0.97      0.97      0.97     49249



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [33]:
with open("kbo_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [34]:
with open("kbo_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)